# Morning Drive Scheduler

### Step 1 - Initialize the script. Run this cell first.

In [ ]:
# Create imports
import pandas as pd
import random
import ipywidgets as widgets
from ipywidgets import interact_manual

# Read excel tables into python
driversOriginal = pd.read_excel('data.xlsx', sheet_name='drivers')
clientsOriginal = pd.read_excel('data.xlsx', sheet_name='clients')
drivers = driversOriginal.copy()
clients = clientsOriginal.copy()
distances = pd.read_excel('data.xlsx', sheet_name='distances')
distances = distances.set_index(distances.columns[0])

# create Arrays of client and driver ids.
clientId = []
for row in clients.iterrows():
    clientId.append(row[1]['id'])
driverId = []
for row in drivers.iterrows():
    driverId.append(row[1]['id'])


# Variables
currentClients = clientId.copy()
currentDrivers = driverId.copy()
schedule = {}

# Creates new tables based on current Clients/Drivers arrayss
def ClientsTable(currentClients, clients):
    newTable = clients[clients["id"].isin(currentClients)]
    return newTable

def DriversTable(driversClients, drivers):
    newTable = drivers[drivers["id"].isin(currentDrivers)]
    return newTable

# Initialize schedule dictionary
for driver in currentDrivers:
    schedule.update({driver:""})

# Creates Checkbox menus
currentClients = clientId.copy()
currentDrivers = driverId.copy()

checkboxesClients = [widgets.Checkbox(value=True, description=label) for label in currentClients]
outputClients = widgets.widgets.GridBox(children=checkboxesClients)

checkboxesDrivers = [widgets.Checkbox(value=True, description=label) for label in currentDrivers]
outputDrivers = widgets.widgets.GridBox(children=checkboxesDrivers)

accordion = widgets.Accordion(children=[outputClients, outputDrivers])
accordion.set_title(0, 'Clients')
accordion.set_title(1, 'Drivers')
print("Step 2 - Uncheck the necessary boxes. Then continue down to the next cell.") 
accordion

### Step 3 - Run the cell below to update Drive Scheduler with your checkbox choices.

In [ ]:
#Code to update the current Drivers and Clients based on the Checkboxes.
currentClients = []
currentDrivers = []
for i in range(0, len(checkboxesClients)):
    if checkboxesClients[i].value == True:
        currentClients = currentClients + [checkboxesClients[i].description]
for i in range(0, len(checkboxesDrivers)):
    if checkboxesDrivers[i].value == True:
        currentDrivers = currentDrivers + [checkboxesDrivers[i].description]
clientFrame = ClientsTable(currentClients, clients)
driverFrame = DriversTable(currentDrivers, drivers)


#Creates a Schedule using a greedy modelled algorithm
def Schedule(currentDrivers, currentClients):
    #initial Variable Creation
    currentDriverTable = DriversTable(currentDrivers, drivers)
    currentClientTable = ClientsTable(currentClients, clients)
    newSchedule = {}
    # Creating holders for original name lists to correct at the end of the function.
    driverCopy = currentDrivers.copy()
    clientCopy = currentClients.copy()
    random.shuffle(driverCopy)
    shortestDistance = None
    shortestClient = None
    #First run through list of drivers. Initializes dictionaries and assigns
    #1 client to each driver until clients list is empty.
    for driver in driverCopy:
        if clientCopy:
            shortestDistance = distances.loc[driver][clientCopy[0]]
            shortestClient = clientCopy[0]
            for client in clientCopy:
                if distances.loc[driver][client] < shortestDistance:
                    shortestDistance = distances.loc[driver][client]
                    shortestClient = client
            if shortestClient:
                newSchedule.update({driver:{shortestClient:shortestDistance}})
        try:
            clientCopy.remove(shortestClient)
            shortestClient = None
            shortestDistance = None
        except:
            pass
    #Runs if there are still clients that need to be assigned after the first 
    #runthrough
    while clientCopy:
        random.shuffle(driverCopy)
        shortestDistance = None
        shortestClient = None
        for driver in driverCopy:
            if clientCopy:
                shortestDistance = distances.loc[driver][clientCopy[0]]
                shortestClient = clientCopy[0]
                for client in clientCopy:
                    if distances.loc[driver][client] < shortestDistance:
                        shortestDistance = distances.loc[driver][client]
                        shortestClient = client
                if shortestClient:
                    temp = newSchedule[driver]
                    temp.update({shortestClient:shortestDistance})
                    newSchedule.update({driver:temp})
            try:
                clientCopy.remove(shortestClient)
                shortestClient = None
                shortestDistance = None
            except:
                pass
    return newSchedule

#returns total distance of a schedule
def TotalMeters(currentSchedule):
    totalMeters = 0
    keys = currentSchedule.keys()
    for key in keys:
        temp = currentSchedule.get(key)
        totalMeters = totalMeters + sum(temp.values())
    return totalMeters

# runs 100 schedules and returns the most optimal one.
def Optimizer(currentDrivers, currentClients):
    iterations = 100
    currentBest = Schedule(currentDrivers,currentClients)
    currentBestMeters = TotalMeters(currentBest)
    for i in range(1,iterations):
        currSchedule = Schedule(currentDrivers, currentClients)
        currMeters = TotalMeters(currSchedule)
        if currMeters < currentBestMeters:
            currentBestMeters = currMeters
            currentBest = currSchedule
    return currentBest, currentBestMeters

def BestRun(currentDrivers, currentClients):
    best, bestMeters = Optimizer(currentDrivers, currentClients)
    print(str(int((bestMeters.item() / 1000))) + "km of distance between client and driver locations (as the crow flies)")
    print("")
    print("")
    print("")
    for driver in best:
        print(driver)
        drives = best[driver].keys()
        for client in drives:
            print("     " + client)

def SingleRun(currentDrivers, currentClients):
    current = Schedule(currentDrivers, currentClients)
    meters = TotalMeters(current)
    print(str(int((meters.item() / 1000))) + "km of distance between client and driver locations (as the crow flies)")
    print("")
    print("")
    print("")
    for driver in current:
        print(driver)
        drives = current[driver].keys()
        for client in drives:
            print("     " + client)

## Step 4

##### Run any of the Drive Scheduler options below. Run them as many times as is necessary to obtain a desired result. SingleRun gives a more varied, less optimal schedule, BestRun gives a less varied and more optimized schedule.

##### To modify the checklist go back to step 2.

### SingleRun

In [ ]:
SingleRun(currentDrivers, currentClients)

### BestRun

In [ ]:
BestRun(currentDrivers, currentClients)